<a href="https://colab.research.google.com/github/mpt777/CS474/blob/main/lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a 
href="https://colab.research.google.com/github/wingated/cs474_labs_f2019/blob/master/DL_Lab6.ipynb"
  target="_parent">
  <img
    src="https://colab.research.google.com/assets/colab-badge.svg"
    alt="Open In Colab"/>
</a>

# Lab 6: Sequence-to-sequence models

### Description:
For this lab, you will code up the [char-rnn model of Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). This is a recurrent neural network that is trained probabilistically on sequences of characters, and that can then be used to sample new sequences that are like the original.

This lab will help you develop several new skills, as well as understand some best practices needed for building large models. In addition, we'll be able to create networks that generate neat text!

### Deliverable:
- Fill in the code for the RNN (using PyTorch's built-in GRU).
- Fill in the training loop
- Fill in the evaluation loop. In this loop, rather than using a validation set, you will sample text from the RNN.
- Implement your own GRU cell.
- Train your RNN on a new domain of text (Star Wars, political speeches, etc. - have fun!)

### Grading Standards:
- 20% Implementation the RNN
- 20% Implementation training loop
- 20% Implementation of evaluation loop
- 20% Implementation of your own GRU cell
- 20% Training of your RNN on a domain of your choice

### Tips:
- Read through all the helper functions, run them, and make sure you understand what they are doing
- At each stage, ask yourself: What should the dimensions of this tensor be? Should its data type be float or int? (int is called `long` in PyTorch)
- Don't apply a softmax inside the RNN if you are using an nn.CrossEntropyLoss (this module already applies a softmax to its input).

### Example Output:
An example of my final samples are shown below (more detail in the
final section of this writeup), after 150 passes through the data.
Please generate about 15 samples for each dataset.

<code>
And ifte thin forgision forward thene over up to a fear not your
And freitions, which is great God. Behold these are the loss sub
And ache with the Lord hath bloes, which was done to the holy Gr
And appeicis arm vinimonahites strong in name, to doth piseling 
And miniquithers these words, he commanded order not; neither sa
And min for many would happine even to the earth, to said unto m
And mie first be traditions? Behold, you, because it was a sound
And from tike ended the Lamanites had administered, and I say bi
</code>


---

## Part 0: Readings, data loading, and high level training

---

There is a tutorial here that will help build out scaffolding code, and get an understanding of using sequences in pytorch.

* Read the following

> * [Pytorch sequence-to-sequence tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) (Take note that you will not be implementing the encoder part of this tutorial.)
* [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)






In [ ]:
! wget -O ./text_files.tar.gz 'https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz' 
! tar -xzf text_files.tar.gz
! pip install unidecode
! pip install torch

import unidecode
import string
import random
import re
 
import pdb
 
all_characters = string.printable
n_characters = len(all_characters)
file = unidecode.unidecode(open('./text_files/lotr.txt').read())
file_len = len(file)
print('file_len =', file_len)

--2021-02-27 17:30:02--  https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz
Resolving piazza.com (piazza.com)... 3.208.138.178, 54.236.183.80, 34.195.100.100, ...
Connecting to piazza.com (piazza.com)|3.208.138.178|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-uploads.piazza.com/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz [following]
--2021-02-27 17:30:03--  https://cdn-uploads.piazza.com/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz
Resolving cdn-uploads.piazza.com (cdn-uploads.piazza.com)... 13.35.90.48, 13.35.90.114, 13.35.90.21, ...
Connecting to cdn-uploads.piazza.com (cdn-uploads.piazza.com)|13.35.90.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1533290 (1.5M) [application/x-gzip]
Saving to: ‘./text_files.tar.gz’

./text_files.tar.gz 100%[===================>]   1.46M  --.-KB/s    in

In [ ]:
chunk_len = 200
 
def random_chunk():
  start_index = random.randint(0, file_len - chunk_len)
  end_index = start_index + chunk_len + 1
  return file[start_index:end_index]
  
print(random_chunk())

nging of swords: such sounds as had not been heard in the hallowed 
places since the building of the City. At last they came to Rath Dnnen and 
hastened towards the House of the Stewards, looming in th


In [ ]:
import torch
# Turn string into list of longs
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
      tensor[c] = all_characters.index(string[c])
  return tensor

print(char_tensor('abcDEF'))

tensor([10, 11, 12, 39, 40, 41])


---

## Part 4: Creating your own GRU cell 

**(Come back to this later - its defined here so that the GRU will be defined before it is used)**

---

The cell that you used in Part 1 was a pre-defined Pytorch layer. Now, write your own GRU class using the same parameters as the built-in Pytorch class does.

Please try not to look at the GRU cell definition. The answer is right there in the code, and in theory, you could just cut-and-paste it. This bit is on your honor!

**TODO:**

**DONE:**
* Create a custom GRU cell



In [88]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class GRU(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):
    super(GRU, self).__init__()

    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers

    self.W_ir = nn.Linear(input_size, hidden_size)
    self.W_hr = nn.Linear(hidden_size, hidden_size)

    self.W_iz = nn.Linear(input_size, hidden_size)
    self.W_hz = nn.Linear(hidden_size, hidden_size)

    self.W_in = nn.Linear(input_size, hidden_size)
    self.W_hn = nn.Linear(hidden_size, hidden_size)

    self.sigmoid = nn.Sigmoid()
    self.tanh = nn.Tanh()
    
  def forward(self, inputs, hidden):
    # Each layer does the following:
    # r_t = sigmoid(W_ir*x_t + b_ir + W_hr*h_(t-1) + b_hr)
    # z_t = sigmoid(W_iz*x_t + b_iz + W_hz*h_(t-1) + b_hz)
    # n_t = tanh(W_in*x_t + b_in + r_t**(W_hn*h_(t-1) + b_hn))
    # h_(t) = (1 - z_t)**n_t + z_t**h_(t-1)
    # Where ** is hadamard product (not matrix multiplication, but elementwise multiplication)

    r_t = self.sigmoid( W_ir(inputs) + W_hr(hidden) )
    z_t = self.sigmoid( W_iz(inputs) + W_hz(hidden) )
    n_t = self.tanh( W_in(inputs) + torch.mul( r_t, W_hn(hidden) ) )

    outputs, hiddens = torch.mul( (1 - z_t), n_t ) + torch.mul(z_t, hidden)
    
    return outputs, hiddens
  


---

##  Part 1: Building a sequence to sequence model

---

Great! We have the data in a useable form. We can switch out which text file we are reading from, and trying to simulate.

We now want to build out an RNN model, in this section, we will use all built in Pytorch pieces when building our RNN class.


**TODO:**
* Create an RNN class that extends from nn.Module.

**DONE:**



In [ ]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, n_layers=1):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.n_layers = n_layers
    
    # more stuff here...
    self.embedding = nn.Embedding(self.input_size, self.hidden_size)
    self.gru = GRU(self.hidden_size, self.hidden_size, self.n_layers)
    self.linear = nn.Linear(self.hidden_size, self.output_size)
    self.relu = nn.ReLU()

  def forward(self, input_char, hidden):
    # by reviewing the documentation, construct a forward function that properly uses the output
    # of the GRU

    # stuff here
    output = self.embedding(input_char).view(1, 1, -1)
    output, hidden = self.gru(output, hidden)
    output = self.linear(output)
    output = self.relu(output)
    return output, hidden

  def init_hidden(self):
    return torch.zeros(self.n_layers, 1, self.hidden_size)

In [ ]:
def random_training_set():    
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp, target

---

## Part 2: Sample text and Training information

---

We now want to be able to train our network, and sample text after training.

This function outlines how training a sequence style network goes. 

**TODO:**

**DONE:**
* Fill in the pieces.




In [ ]:
# NOTE: decoder_optimizer, decoder, and criterion will be defined below as global variables
def train(inp, target):
  ## initialize hidden layers, set up gradient and loss 
    # your code here
  ## /
  decoder_optimizer.zero_grad()
  hidden = decoder.init_hidden()
  loss = 0
  
  for feature_char, target_char in zip(inp, target):
    
    decoded_char, hidden = decoder(feature_char, hidden)
    loss += criterion(decoded_char.squeeze(0), target_char.unsqueeze(0))
    
  loss.backward()
  decoder_optimizer.step()
  
  return loss.item()/len(inp)

---

## Part 3: Sample text and Training information

---

You can at this time, if you choose, also write out your train loop boilerplate that samples random sequences and trains your RNN. This will be helpful to have working before writing your own GRU class.

If you are finished training, or during training, and you want to sample from the network you may consider using the following function. If your RNN model is instantiated as `decoder`then this will probabilistically sample a sequence of length `predict_len`

**TODO:**

**DONE:**
* Fill out the evaluate function to generate text frome a primed string



In [ ]:
def sample_outputs(output, temperature):
    """Takes in a vector of unnormalized probability weights and samples a character from the distribution"""
    return torch.multinomial(torch.exp(output / temperature), 1)

def evaluate(prime_str='A', predict_len=100, temperature=0.8):
  ## initialize hidden state, initialize other useful variables
  hidden = decoder.init_hidden()
  prime_char = char_tensor(prime_str)

  # your code here
  ## /
  evaluated_char = prime_char[-1]

  prediction = prime_str
  
  for i in range(predict_len):
    out, hidden = decoder(evaluated_char, hidden)
    probability_distribution = out.data.view(-1).div(temperature).exp()
    select_ind = torch.multinomial(probability_distribution, 1)[0]

    predicted_char = all_characters[select_ind]
    evaluated_char = char_tensor(predicted_char)
    prediction += predicted_char

  return prediction

---

## Part 4: (Create a GRU cell, requirements above)

---



---

## Part 5: Run it and generate some text!

---


**TODO:** 


**DONE:**
* Create some cool output




Assuming everything has gone well, you should be able to run the main function in the scaffold code, using either your custom GRU cell or the built in layer, and see output something like this. I trained on the “lotr.txt” dataset, using chunk_length=200, hidden_size=100 for 2000 epochs. These are the results, along with the prime string:

---

 G:
 
 Gandalf was decrond. 
'All have lord you. Forward the road at least walk this is stuff, and 
went to the long grey housel-winding and kindled side was a sleep pleasuring, I do long 
row hrough. In  

 lo:
 
 lost death it. 
'The last of the gatherings and take you,' said Aragorn, shining out of the Gate. 
'Yes, as you there were remembaused to seen their pass, when? What 
said here, such seven an the sear 

 lo:
 
 low, and frod to keepn 
Came of their most. But here priced doubtless to an Sam up is 
masters; he left hor as they are looked. And he could now the long to stout in the right fro horseless of 
the like 

 I:
 
 I had been the 
in his eyes with the perushed to lest, if then only the ring and the legended 
of the less of the long they which as the 
enders of Orcovered and smood, and the p 

 I:
 
 I they were not the lord of the hoomes. 
Home already well from the Elves. And he sat strength, and we 
housed out of the good of the days to the mountains from his perith. 

'Yess! Where though as if  

 Th:
 
 There yarden 
you would guard the hoor might. Far and then may was 
croties, too began to see the drumbred many line 
and was then hoard walk and they heart, and the chair of the 
Ents of way, might was 

 G:
 
 Gandalf 
been lat of less the round of the stump; both and seemed to the trees and perished they 
lay are speered the less; and the wind the steep and have to she 
precious. There was in the oonly went 

 wh:
 
 which went out of the door. 
Hull the King and of the The days of his brodo 
stumbler of the windard was a thing there, then it been shining langing 
to him poor land. They hands; though they seemed ou 

 ra:
 
 rather,' have all the least deather 
down of the truven beginning to the house of sunk. 
'Nark shorts of the Eyes of the Gate your great nothing as Eret. 
'I wander trust horn, and there were not, it  

 I:
 
 I can have no mind 
together! Where don't may had one may little blung 
terrible to tales. And turn and Gandalf shall be not to as only the Cattring 
not stopped great the out them forms. On they she lo 

---


In [ ]:
import time
n_epochs = 5000
print_every = 200
plot_every = 10
hidden_size = 200
n_layers = 1
lr = 0.001
 
decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
 
start = time.time()
all_losses = []
loss_avg = 0

In [ ]:
# n_epochs = 2000
for epoch in range(1, n_epochs + 1):
  loss_ = train(*random_training_set())       
  loss_avg += loss_

  if epoch % print_every == 0:
      print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
      print(evaluate('Wh', 100), '\n')

  if epoch % plot_every == 0:
      all_losses.append(loss_avg / plot_every)
      loss_avg = 0

[62.45153594017029 (200 4%) 2.2668]
Whil, xode farir so, tue a loud sen 'ser 
aode sto oug and ewin . I herDt enmengoot, meates eir mathe  

[125.10501456260681 (400 8%) 2.2142]
Wht, ;ourded to the stit, &oy and and as ofly ree stare \andons aller, ffor wreter. 'I was allee to fo 

[188.2317395210266 (600 12%) 2.0372]
Wh!' leasine 
the to this allo
 a from soind sforther the of the sestist the mopts thing 
grathes what 

[250.73269081115723 (800 16%) 1.9768]
What we parch 
and now the shood that a not from there liked to the doow misted that his 
word to the  

[313.2021861076355 (1000 20%) 1.5943]
When and not us. It will king. 


He said nothery of 
the Alleed under the 

of mached 
overs. 

'Leve 

[375.53284096717834 (1200 24%) 1.8593]
Whe over a him. 

A lorthed. 'Welvering intode 
of the wormen 
purued. ' 

'A said like enemore, and 
 

[437.55659317970276 (1400 28%) 1.4815]
Wher that 
said Iwayter. ' he were was them to the 
doused and was I sproms and he mast cloant,' she s 

[

In [ ]:
for i in range(10):
  start_strings = [" Th", " wh", " he", " I ", " ca", " G", " lo", " ra"]
  start = random.randint(0,len(start_strings)-1)
  print(start_strings[start])
#   all_characters.index(string[c])
  print(evaluate(start_strings[start], 200), '\n')

 Th
 The ton to done. They had - on the 
for uffer to his once. 



The found out of the darkness of 
the time. He shoulder passing, 
and all the dark were may stood, and the way with the ringly in the will  

 ra
 ragoring old stoor road this 
at mon, and this guess 
the lasts of this folk the treached. 

Then 2 might. know more they what's it recharth, 
and tather did and were noits upon their speak while what m 

 G
 Gall Frodo and do 5or you will slowed to 
which 
the morning you can to the on!' Xently the dirs: do not silent on the 3)`k, for Pippin, thought the 
dright, Denound and 
his pon to me. I've recasing w 

 wh
 whe or hope; for the trest would 
ardel in the little higher would for the moming more the cumpasted from the rasves who were have a twight; and it was ta 

 he
 her ~imlit. He some, and he 
have march in the Xessained there was nothing, and going at one with the 
will falles, he said, and you don't dark than a 
pires-found they had some or 
soon at have not wer 



---

## Part 6: Generate output on a different dataset

---

**TODO:**


**DONE:**
* Choose a textual dataset. Here are some [text datasets](https://www.kaggle.com/datasets?tags=14104-text+data%2C13205-text+mining) from Kaggle 

* Generate some decent looking results and evaluate your model's performance (say what it did well / not so well)



In [85]:
import pandas as pd

text = pd.read_csv("./Costumes_Amazon.csv", error_bad_lines=False)

review_text = text.iloc[:,0]

final_text = ' '.join(review_text.tolist())


myText = open(r'/reviews.txt','w')
myText.write(final_text)
myText.close()

In [90]:
all_characters = string.printable
n_characters = len(all_characters)
file = unidecode.unidecode(open('./reviews.txt').read())
file_len = len(file)

chunk_len = 200
 
def random_chunk():
  start_index = random.randint(0, file_len - chunk_len)
  end_index = start_index + chunk_len + 1
  return file[start_index:end_index]
  
print(random_chunk())

def random_training_set():    
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp, target

on my head to fill in space. Out of all the costumes of this style, this is the best.
 
  I ordered this for my husband. He was kind of thrown off by the trim which is a bright gold. But, overall it's 


In [92]:
# n_epochs = 2000
for epoch in range(1, n_epochs + 1):
  loss_ = train(*random_training_set())       
  loss_avg += loss_

  if epoch % print_every == 0:
      print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
      print(evaluate('T', 100), '\n')

  if epoch % plot_every == 0:
      all_losses.append(loss_avg / plot_every)
      loss_avg = 0

[1614462539.893113 (200 4%) 1.2474]
The man plain car compantally in the price part to odd and the sleeves and came ealisy good.
 
  They 

[1614462602.4062767 (400 8%) 1.8041]
To did tords perfect and its pero real up and didn't care at a really downsing legs with my 1stame an 

[1614462665.1837597 (600 12%) 1.0432]
THI'ST I ears danding.
 
  Though, noise it car [under this costume for my daughter is super and it w 

[1614462727.0978496 (800 16%) 1.3695]
The costume has spra(s it and they stretchy pieces and it fit perfect from and perfect but still fit  

[1614462789.2747843 (1000 20%) 1.2221]
To. It's a little ^sall and it fits performy for the colder than a Halloween costume.  My son loved t 

[1614462851.3359609 (1200 24%) 1.4311]
TOK, it's will #lreaded for a 3T fit...
 
  {uck head you my little long and he's I end and it fits m 

[1614462913.4324393 (1400 28%) 1.5050]
The costuments head out so I wear a tail she will <rd On used to too laughter so it was for when is s 

[16144

For my text, I chose a kaggle dataset of Amazon reviews for halloween costumes. Very fun to play around with! 

The model was able to produce fairly good results! There still were issues with random characters, spelling, and symbols, but the setiment was definitely correct! I think the final line of the model "I loved it" was the perfect way to close off the project 